In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import math
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# PyTorch libraries and modules
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout


from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


# self write tools
from tools import LABEL_ENCODER, transforms_train, transforms_test
from models import CNN
from dataset import CnnDataset, TestDataset

# Check GPU

In [2]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("Using {}: {}".format(device, torch.cuda.get_device_name(0)))

Using cuda: GeForce RTX 3080


# Global Variables

In [3]:
IMAGES_DIR = 'input/images/'
TRAIN_CSV = 'input/split/train.csv'
VALID_CSV = 'input/split/valid.csv'

BATCH_SIZE = 256
EPOCHS = 20





# Load Data

In [4]:
def create_cnn_iterator(csv_path):
    df = pd.read_csv(csv_path)

    X = list(df['id'])
    y = list(df['category'])

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=11)

    y_train, y_valid = LABEL_ENCODER.transform(y_train), LABEL_ENCODER.transform(y_valid)


    train_dataset = CnnDataset(transforms_train, X_train, y_train)
    test_dataset = CnnDataset(transforms_test, X_valid, y_valid)

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    return train_dataloader, test_dataloader

train_dataloader, test_dataloader = create_cnn_iterator("input/train.csv")

# 2. Model

In [5]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device, dtype=torch.int64)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_cnn(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for (x, y) in iterator:
            
            x = x.to(device)
            y = y.to(device, dtype=torch.int64)
                    
            y_pred = model(x)
            
            loss = criterion(y_pred, y)
            
            acc = calculate_accuracy(y_pred, y)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [6]:
model = CNN()
optimizer = Adam(model.parameters(), lr=0.0001)
criterion = CrossEntropyLoss()

model = model.cuda()
criterion = criterion.cuda()


In [7]:





# train_accuracy_list = []
# train_loss_list = []
# valid_acc_list = []
# valid_loss_list = []
# best_valid_loss = float('inf')

# for epoch in range(EPOCHS):

#     start_time = time.monotonic()

#     # train
#     train_loss, train_acc = train(model, train_dataloader, optimizer, criterion, device)
    
#     # valid
#     valid_loss, valid_acc = evaluate_cnn(model, test_dataloader, criterion, device)

#     # save best model
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'output/cnn-model.pt')

#     # Track the accuracy
#     train_accuracy_list.append(train_acc)
#     train_loss_list.append(train_loss)
#     valid_acc_list.append(valid_acc)
#     valid_loss_list.append(valid_loss)
        
#     # print epoch info
#     end_time = time.monotonic()
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#     print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        




In [8]:
model.load_state_dict(torch.load('output/cnn-model.pt'))
test_loss, test_acc = evaluate_cnn(model, test_dataloader, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.245 | Test Acc: 92.82%


In [15]:
test_df = pd.read_csv('input/test.csv')

test_dataset = TestDataset(test_df)

dataloader = DataLoader(test_dataset, batch_size=1024)

df = pd.DataFrame()
model.eval()
    
with torch.no_grad():

    for (ids, images) in tqdm(dataloader):
            
        images = images.to(device)
        
                    
        y_pred = model(images)
        

        top_pred = y_pred.argmax(1).cpu()
        labels = LABEL_ENCODER.inverse_transform(top_pred)

        temp = pd.DataFrame()
        temp['id'] = ids
        temp['category'] = labels
        

        df = df.append(temp)


print(df)
df.to_csv('output/cnn_prediction.csv', index=False)


100%|██████████| 22/22 [00:11<00:00,  1.88it/s]        id   category
0    26266      Dress
1    22134    Topwear
2    28358      Belts
3    15554      Shoes
4    53408  Innerwear
..     ...        ...
119  39737    Eyewear
120  57477      Dress
121  22312    Topwear
122  54105       Lips
123  14080       Bags

[21628 rows x 2 columns]

